# 0: Import packages

In [ ]:
from PIL import Image, ImageFile #pip install Pillow==9.4.0
import sys
import os
import numpy as np
import logging
import glob
import subprocess
import json
import pandas as pd

from ImageCrop import ImagePreprocessor
from SpotterWrapper import Spotter, PolygonVisualizer
from IPython.display import display
from shapely.geometry import Polygon

logging.basicConfig(level=logging.INFO)
Image.MAX_IMAGE_PIXELS=None
ImageFile.LOAD_TRUNCATED_IMAGES = True


# 1: Specify filepaths

In [ ]:
# Name folders for raw data and processed data
map_data_topfolder = 'raw_maps_20231024'
map_strec_topfolder = 'processed/strec'

for fp in [map_strec_topfolder]:
    if not os.path.isdir(fp):
        os.makedirs(fp)

# IMPORTANT! Locate spotter directory and detectron weights
git_clone_location = 'C:/repo/'
spotter_directory = git_clone_location + 'mapkurator-spotter/spotter-v2'
model_weights = git_clone_location + 'detectron2-master/detectron2/checkpoint/model_v2_en.pth'
spotter_config = spotter_directory + '/configs/PALEJUN/Finetune/Rumsey_Polygon_Finetune.yaml'

# 2: Crop all jpeg maps in (user defined) map_data_topfolder

In [ ]:
def pyramid_scan(img_path, output_dir, save_each_layer=False):
    image = Image.open(img_path)
    image_preprocessor = ImagePreprocessor(image, overlapping_tolerance=0.3, num_layers=5, min_patch_resolution=512, max_patch_resolution=4096)
    image_preprocessor.process()
    print("preprocessing done")
    spotter = Spotter(spotter_config, model_weights, confidence_thresh=0.7, draw_thresh=0.85)
    all_layer_results = []

    base_image_batch, base_offset_xs, base_offset_ys = image_preprocessor.get_image_patches(0)
    vis = PolygonVisualizer()
    vis.canvas_from_patches(base_image_batch, base_offset_xs, base_offset_ys)

    for i in range(image_preprocessor.num_layers):
        # If you want to save for each layer, uncomment the following line
        # image_preprocessor.save_patches(os.path.join(output_dir, f'layer_{i}_patches'), layer=i)

        image_batch, offset_xs, offset_ys = image_preprocessor.get_image_patches(i)
        spotter.load_batch(image_batch, offset_xs, offset_ys)
        results = spotter.inference_batch()
        all_layer_results.extend(results)

        #all_layer_offset_xs.extend(offset_xs)
        #all_layer_offset_ys.extend(offset_ys)

        if save_each_layer == True:
            vis.draw(results).save(os.path.join(output_dir, f'combined_tagged_{i}.png'))
            vis.save_json(results, os.path.join(output_dir, f'combined_tagged_{i}.json'))
        else:
            pass

    vis.draw(all_layer_results).save(os.path.join(output_dir, f'combined_tagged_all_layers.png'))
    vis.save_json(all_layer_results, os.path.join(output_dir, f'combined_tagged_all_layers.json'))

# Run crop on all maps
for map_data_subfolder in next(os.walk(map_data_topfolder))[1]:
    jpeg_list = glob.glob(map_data_topfolder + '/' + map_data_subfolder + '/*.jpeg')
    if len(jpeg_list) != 1:
        print(map_data_subfolder + " failed. Please ensure there is exactly 1 file with extension .jpeg in the folder.")
    else:
        map_image = jpeg_list[0].split("\\")[1]
        if map_data_subfolder in ['1846_vandevelde', '1874_saunders', '1845_kiepert']: # '1858_vandevelde', '1874_saunders', '1845_kiepert']: #,,]: #'1858_vandevelde', '1847_tobler', '1845_kiepert'
            img_path = map_data_topfolder + '/' + map_data_subfolder + "/" + map_image
            map_name = os.path.basename(img_path).split('.')[0] # get the map name without extension
            output_dir = os.path.join(map_strec_topfolder, map_name)
            if not os.path.isdir(output_dir):
                os.makedirs(output_dir)
            pyramid_scan(img_path, output_dir, save_each_layer=False)
            logging.info('Done cropping %s' %img_path )

# 3: Label Combination

In [1]:
from PIL import Image, ImageFile
import json 
import pandas as pd
from collections import Counter
from shapely.geometry import Polygon, MultiPolygon
from itertools import combinations

import numpy as np
import importlib
import Clustering
import TextRectify
import TextAmalgamate
import ExtractHandling
import json
import pickle
import SpotterWrapper
import Grouping
import BezierSplineMetric
import FontSimilarity
import SequenceRecovery
import SubwordDeduplication as sd
importlib.reload(SpotterWrapper)
importlib.reload(Grouping)
importlib.reload(Clustering)
importlib.reload(TextRectify)
importlib.reload(TextAmalgamate)
importlib.reload(ExtractHandling)
importlib.reload(BezierSplineMetric)
importlib.reload(FontSimilarity)
importlib.reload(SequenceRecovery)

map_name_in_strec = 'vandevelde_1846' # 'kiepert_1845', 'saunders_1874', 'vandevelde_1846'

c:\Users\26553\anaconda3\envs\mapKurator\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
Using device: cuda
Using device: cuda
Using device: cuda


## 3.1 Subword Deduplication

In [11]:
sd.subword_deduplication(map_name_in_strec, do_cluster_pre_merge=True)

971 polygons kept.


## 3.2 Nested Word Flattening

In [12]:
# Amalgamation stage - assumes there exists "combined_tagged_all_layers_rectified_premerge.json" in map_name_in_strec processed folder.
df = ExtractHandling.prepare_labels_for_amalgamation(map_name_in_strec)
df = TextAmalgamate.amalgamate_labels_wrapper(df, 0.75, .5)

# Save amalgamated labels
with open(f'processed/strec/{map_name_in_strec}/deduplicated_flattened_labels.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)

c:\Users\26553\anaconda3\envs\mapKurator\lib\site-packages\pandas\core\dtypes\cast.py:1983: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  result[:] = values


971 labels.
894 labels.
871 labels.
868 labels.
867 labels.
Amalgamation completed with 867 labels.


## 3.3 Multi-Word Sequence Recovery

### 3.3.1 Prepare by calculating spline and font metrics

In [3]:
df = pickle.load(open('processed/strec/' + map_name_in_strec + '/deduplicated_flattened_labels.pickle', 'rb'))
df['polygons'] = df['labels'].apply(lambda x: x[0])
df['texts'] = df['labels'].apply(lambda x: x[1])

# Uncomment to draw splines later
## BezierSplineMetric.draw_splines(map_name_in_strec, polygons, texts, PCA_features, all_splines)

# reset index so list-based operations match df index
df = df.reset_index(drop=True).copy()

# pca for principal directions
df['PCA_features'] = Grouping.calc_PCA_feats(df['polygons'], do_separation=True, enhance_coords=True)

# find neighbors for spline and font metric consideration
df = BezierSplineMetric.calc_neighbours(df, radius_multiplier = 40)

# calculate spline metric between identified neighbors
df = BezierSplineMetric.spline_metric(df)

# calculate font metric between identified neighbors - long due to need to work with images
df = FontSimilarity.calc_font_similarities(df, map_name_in_strec)

C:\Users\HP\AppData\Local\Temp\ipykernel_21936\620548058.py:1: UserWarning: Unpickling a shapely <2.0 geometry object. Please save the pickle again; shapely 2.1 will not have this compatibility.
  df = pickle.load(open('processed/strec/' + map_name_in_strec + '/deduplicated_flattened_labels.pickle', 'rb'))


In [4]:
with open(f'processed/strec/{map_name_in_strec}/seq_rec_prepared_labels.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [133]:
# Optional - draw splines
#df = pickle.load(open(f'processed/strec/{map_name_in_strec}/seq_rec_prepared_labels.pickle', 'rb'))
#BezierSplineMetric.draw_splines(map_name_in_strec, df['polygons'].tolist(), df['texts'].tolist(), df['PCA_features'].tolist(), df['all_splines'].explode().dropna().tolist(), spline_metric_threshold = 0.01)

### 3.3.2 Iterative Sequence Recovery

In [8]:
df = pickle.load(open(f'processed/strec/{map_name_in_strec}/seq_rec_prepared_labels.pickle', 'rb'))
# Drop PCA_features - no longer needed, makes me feel good to discard stuff i don't need
df.drop('PCA_features', axis=1, inplace=True)
df.drop('all_splines', axis=1, inplace=True)

In [9]:
df = SequenceRecovery.sl_sequence_recovery_wrapper(df, font_threshold=.0, bezier_threshold=.5)

867 labels.
617 labels.
538 labels.
523 labels.
522 labels.
Sequence Recovery completed with 522 labels.


In [10]:
# Save amalgamated labels
with open(f'processed/strec/{map_name_in_strec}/fully_processed_labels.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)

# 4: Evaluation

In [6]:
from PIL import Image, ImageFile
import pandas as pd
from itertools import combinations
import scipy
import numpy as np
import importlib 

import Evaluation
importlib.reload(Evaluation)
%load_ext autoreload

## 4.1: Isolate crops to be used for evaluation

In [101]:

def visualize_crop(map_name_in_strec, raw_or_spotter, left_x, right_x, top_y, bottom_y):
    if raw_or_spotter == "raw":
        map_img = Image.open('processed/strec/' + map_name_in_strec + '/raw.jpeg') 
    elif raw_or_spotter == "spotter_0":
        map_img = Image.open('processed/strec/' + map_name_in_strec + '/combined_tagged_0.png')
    elif raw_or_spotter == "spotter_1":
        map_img = Image.open('processed/strec/' + map_name_in_strec + '/combined_tagged_1.png')
    elif raw_or_spotter == "spotter_2":
        map_img = Image.open('processed/strec/' + map_name_in_strec + '/combined_tagged_2.png')
    elif raw_or_spotter == "all":
        map_img = Image.open('processed/strec/' + map_name_in_strec + '/combined_tagged_all_layers.png')
    elif raw_or_spotter == "rectified":
        map_img = Image.open('processed/strec/' + map_name_in_strec + '/combined_tagged_all_layers_rectified.png')
    width, height = map_img.size
    print("full map is " + str(width) + " pixels wide by " + str(height) + " pixels high.\n displaying crop:")
    display(map_img.crop((left_x, top_y, right_x, bottom_y, )))

left_x = 2475
right_x = 3550
top_y = 4820
bottom_y = 5850

#visualize_crop("kiepert_1845", "all", left_x, right_x, top_y, bottom_y)

## 4.2 Precision and Recall: 1:1 Matching on Geometry, then IoU

In [12]:
kiepert_gt_patches = [[1750, 3750, 4775, 6200], [2250, 4050, 6050, 7500]] #,[2475, 3550, 4820, 5850]]
saunders_gt_patches = [[2350, 3850, 1750, 3250], [6450, 7500, 2200, 3250], [5400, 6400, 4500, 5500], [7650, 8650, 5400, 6400], [7650, 8650, 3150, 4150]] #
vandevelde_gt_patches = [[2850, 5250, 1450, 3850]]

# Gimme them numbers :)

kname = "kiepert_1845"
vname = "vandevelde_1846"
sname = "saunders_1874"

multiline_handling = "components" # "largest" for multiline gt

print("\nkiepert baseline (gt = " + multiline_handling + ")\n")
kbase_geo_prec, kbase_text_prec, kbase_geo_rec, kbase_text_rec, kbase_IoU_pairs, kbase_num_detected, kbase_num_gt = Evaluation.prec_rec(kname, multiline_handling, kiepert_gt_patches, "methods_0")
print("\nkiepert pyramid - subword dedup, nested word flattening (gt = " + multiline_handling + ")\n")
k12_geo_prec, k12_text_prec, k12_geo_rec, k12_text_rec, k12_IoU_pairs, k12_num_detected, k12_num_gt = Evaluation.prec_rec(kname, multiline_handling, kiepert_gt_patches, "methods_1_2")
print("\nkiepert pyramid - subword dedup, nested word flattening, sequence recovery (gt = " + multiline_handling + ")\n")
k123_geo_prec, k123_text_prec, k123_geo_rec, k123_text_rec, k123_IoU_pairs, k123_num_detected, k123_num_gt = Evaluation.prec_rec(kname, multiline_handling, kiepert_gt_patches, "methods_1_2_3")

print("\nvandevelde baseline (gt = " + multiline_handling + ")\n")
vbase_geo_prec, vbase_text_prec, vbase_geo_rec, vbase_text_rec, vbase_IoU_pairs, vbase_num_detected, vbase_num_gt = Evaluation.prec_rec(vname, multiline_handling, vandevelde_gt_patches, "methods_0")
print("\nvandevelde pyramid - subword dedup, nested word flattening (gt = " + multiline_handling + ")\n")
v12_geo_prec, v12_text_prec, v12_geo_rec, v12_text_rec, v12_IoU_pairs, v12_num_detected, v12_num_gt = Evaluation.prec_rec(vname, multiline_handling, vandevelde_gt_patches, "methods_1_2")
print("\nvandevelde pyramid - subword dedup, nested word flattening, sequence recovery (gt = " + multiline_handling + ")\n")
v123_geo_prec, v123_text_prec, v123_geo_rec, v123_text_rec, v123_IoU_pairs, v123_num_detected, v123_num_gt = Evaluation.prec_rec(vname, multiline_handling, vandevelde_gt_patches, "methods_1_2_3")

print("\nsaunders baseline (gt = " + multiline_handling + ")\n")
sbase_geo_prec, sbase_text_prec, sbase_geo_rec, sbase_text_rec, sbase_IoU_pairs, sbase_num_detected, sbase_num_gt = Evaluation.prec_rec(sname, multiline_handling, saunders_gt_patches, "methods_0")
print("\nsaunders pyramid - subword dedup, nested word flattening (gt = " + multiline_handling + ")\n")
s12_geo_prec, s12_text_prec, s12_geo_rec, s12_text_rec, s12_IoU_pairs, s12_num_detected, s12_num_gt = Evaluation.prec_rec(sname, multiline_handling, saunders_gt_patches, "methods_1_2")
print("\nsaunders pyramid - subword dedup, nested word flattening, sequence recovery (gt = " + multiline_handling + ")\n")
s123_geo_prec, s123_text_prec, s123_geo_rec, s123_text_rec, s123_IoU_pairs, s123_num_detected, s123_num_gt = Evaluation.prec_rec(sname, multiline_handling, saunders_gt_patches, "methods_1_2_3")


kiepert baseline (gt = components)

Avg of Geographic Precision: 0.12549173691123175
Avg of Text Precision: 0.10223378168465885
Avg of Geographic Recall: 0.47456070625052005
Avg of Text Recall: 0.3866082088994571

kiepert pyramid - subword dedup, nested word flattening (gt = components)

Avg of Geographic Precision: 0.23228497443998528
Avg of Text Precision: 0.21090417116489418
Avg of Geographic Recall: 0.5339884469884719
Avg of Text Recall: 0.4848371750917108

kiepert pyramid - subword dedup, nested word flattening, sequence recovery (gt = components)

Avg of Geographic Precision: 0.4265777313288001
Avg of Text Precision: 0.4128542063038088
Avg of Geographic Recall: 0.5932862700090208
Avg of Text Recall: 0.5741995283075962

vandevelde baseline (gt = components)

Avg of Geographic Precision: 0.1180060731297011
Avg of Text Precision: 0.1226172578700649
Avg of Geographic Recall: 0.4895996651125897
Avg of Text Recall: 0.5087311762694182

vandevelde pyramid - subword dedup, nested word fl

FileNotFoundError: [Errno 2] No such file or directory: 'processed/strec/saunders_1874/fully_processed_labels.pickle'

## 5: Plot and Save results

In [11]:
import shapely as sh

# Plot the final image
Evaluation.plot_recovered_seq(map_name_in_strec)

# Save final results into a json file
spotter_labels_full = ExtractHandling.load_processed_labels(map_name_in_strec, "methods_1_2_3")
polygon_list = spotter_labels_full['label_polygons'].tolist()
text_list = spotter_labels_full['annotation'].tolist()

polygons_json = []
for poly, text in zip(polygon_list, text_list):
    dict = {'polygon_x': [], 'polygon_y': [], 'text': text}
    if isinstance(poly, sh.geometry.polygon.Polygon):
        dict['polygon_x'] = list(poly.exterior.coords.xy[0])
        dict['polygon_y'] = list(poly.exterior.coords.xy[1])
            
    elif isinstance(poly, sh.geometry.multipolygon.MultiPolygon):
        for p in poly.geoms: # kaede added .geoms - package version differences
            polygon_x = p.exterior.coords.xy[0]
            polygon_y = p.exterior.coords.xy[1]

            for x, y in zip(polygon_x, polygon_y):
                dict['polygon_x'].append(x)
                dict['polygon_y'].append(y)

    polygons_json.append(dict)

with open(f'processed/strec/{map_name_in_strec}/final.json', 'w') as f:
    json.dump(polygons_json, f)